In [ ]:
# LLM tentative

In [ ]:
!pip install tensorflow

In [ ]:
!pip install transformers

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
# import section

import yfinance as yf
import numpy as np
import torch
import matplotlib.pyplot as plt
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TFGPT2LMHeadModel

In [ ]:
import torch
x = torch.rand(5, 3)
print(x)

In [ ]:
import torch
torch.cuda.is_available()

In [67]:
ticker = "AAPL"
start_date = "2023-01-01"
end_date = "2023-06-08"
data = yf.download(ticker, start=start_date, end=end_date)

prices = data["Close"].tolist()

[*********************100%%**********************]  1 of 1 completed


In [64]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-02,18131.699219,18215.150391,18086.500000,18197.449219,18197.449219,256100
2023-01-03,18163.199219,18251.949219,18149.800781,18232.550781,18232.550781,208700
2023-01-04,18230.650391,18243.000000,18020.599609,18042.949219,18042.949219,235200
2023-01-05,18101.949219,18120.300781,17892.599609,17992.150391,17992.150391,269900
2023-01-06,18008.050781,18047.400391,17795.550781,17859.449219,17859.449219,238200
...,...,...,...,...,...,...
2023-06-01,18579.400391,18580.300781,18464.550781,18487.750000,18487.750000,265600
2023-06-02,18550.849609,18573.699219,18478.400391,18534.099609,18534.099609,270900
2023-06-05,18612.000000,18640.150391,18582.800781,18593.849609,18593.849609,256100


Tokenizing Historical Prices and Fine-tuning the Model

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
# model = GPT2LMHeadModel.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2") # Using TensorFlow instead of PyTorch

In [ ]:
# optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
def tokenize():
    encoded_prices = tokenizer.encode(" ".join([str(price) for price in prices]), return_tensors="pt")

tokenize()




In [ ]:
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
model.resize_token_embeddings(len(tokenizer))



fine-tune the GPT-2 model

In [ ]:
for _ in range(3):
    model.zero_grad()
    outputs = model(encoded_prices, labels=encoded_prices)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

Using historical stock price data, this code refines a pre-trained GPT model to
produce future prices.

In [ ]:
generated = model.generate(encoded_prices, max_length=len(encoded_prices) + 10, temperature=1.0, num_return_sequences=1)
generated_prices = tokenizer.decode(generated[0], skip_special_tokens=True).split()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(data.index, prices, label="Historical Prices")
plt.plot(data.index[-1] + pd.to_timedelta(np.arange(1, len(generated_prices) + 1), 'D'), [float(price) for price in generated_prices[len(prices):]], "g^", label="Predicted Prices")
plt.xlabel("Date")
plt.ylabel("Stock Price")
plt.title(f"{ticker} - Historical and Predicted Stock Prices (GPT)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()